In [1]:
from huggingface_hub import InferenceClient
from tqdm import tqdm
import os
import json

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
from concurrent.futures import ThreadPoolExecutor, as_completed

generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=4096,
    top_p=0.95,
    repetition_penalty=1.0,
    do_sample=True,
)

In [3]:
client = InferenceClient(
    "", timeout = 120
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

In [4]:
!mkdir mixtral-conversation-disagree
# !rm mixtral-factual-wrong-v2/*.json

mkdir: cannot create directory ‘mixtral-conversation-disagree’: File exists


In [5]:
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/amanz.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/asklegal.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/autobuzz.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/bidadari.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/blog.limkitsiang.com.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/blogtipskerjaya.net.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/buro247.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/doctoroncall.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/ecentral.my.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/gov.my.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/majalahsains.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/majalahpama.my.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/mygameon.my.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/nasilemaktech.com.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/pokde.net.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/patriots.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/ringgitohringgit.com.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/rootofscience.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/sukanz.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/vocket.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/alhijrahnews-articles.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/dewanbahasa-jdbp.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/e-penerbitan.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/fiksyenshasha.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/edu.my.jsonl
# !wget https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/resolve/main/hansard.jsonl

In [6]:
files = [
    'amanz.jsonl',
    'asklegal.jsonl',
    'autobuzz.jsonl',
    'bidadari.jsonl',
    'blog.limkitsiang.com.jsonl',
    'blogtipskerjaya.net.jsonl',
    'buro247.jsonl',
    'doctoroncall.jsonl',
    'ecentral.my.jsonl',
    'gov.my.jsonl',
    'majalahsains.jsonl',
    'majalahpama.my.jsonl',
    'mygameon.my.jsonl',
    'nasilemaktech.com.jsonl',
    'pokde.net.jsonl',
    'patriots.jsonl',
    'ringgitohringgit.com.jsonl',
    'rootofscience.jsonl',
    'sukanz.jsonl',
    'vocket.jsonl',
    'alhijrahnews-articles.jsonl',
    'dewanbahasa-jdbp.jsonl',
    'e-penerbitan.jsonl',
    'fiksyenshasha.jsonl',
    'edu.my.jsonl',
    'hansard.jsonl'
]

In [7]:
texts = []
for f in files:
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            if len(l.split()) < 150:
                continue
            texts.append(l)
len(texts)

379790

In [8]:
prompts = []
for t in texts:
    prompt = f'{t}\n\ngenerate questions based on context above'
    prompts.extend([(t, prompt)] * 1)
    
len(prompts)

379790

In [9]:
!mkdir mixtral-disagree-conversation

mkdir: cannot create directory ‘mixtral-disagree-conversation’: File exists


In [10]:
def answer(q, i):
    filename = f'mixtral-disagree-conversation/{i}.json'
    if os.path.exists(filename):
        return
    
    while True:
        try:
            prompt = q[1]
            formatted_prompt = format_prompt(prompt, [])
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            with open(filename, 'w') as fopen:
                json.dump((q[0], output), fopen)
            break
        except Exception as e:
            print(e)
            pass

In [ ]:
max_worker = 50

questions = prompts
for i in tqdm(range(0, len(questions), max_worker)):
    urls_ = [(q, no + i) for no, q in enumerate(questions[i: i + max_worker])]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(answer, url[0], url[1]): url for url in urls_}

        for future in as_completed(futures):
            future.result()

 12%|████████████▎                                                                                           | 897/7596 [17:55<75:07:18, 40.37s/it]

504 Server Error: Gateway Time-out for url: https://mixtral.us-west-2.mesolitica.com/
504 Server Error: Gateway Time-out for url: https://mixtral.us-west-2.mesolitica.com/


 13%|█████████████▏                                                                                          | 963/7596 [45:31<44:14:54, 24.02s/it]

504 Server Error: Gateway Time-out for url: https://mixtral.us-west-2.mesolitica.com/


 13%|█████████████▏                                                                                          | 964/7596 [46:51<75:24:52, 40.94s/it]

...

 19%|███████████████████▎                                                                                 | 1457/7596 [3:39:20<27:26:28, 16.09s/it]

504 Server Error: Gateway Time-out for url: https://mixtral.us-west-2.mesolitica.com/


 25%|█████████████████████████▏                                                                           | 1896/7596 [6:05:06<31:01:03, 19.59s/it]

504 Server Error: Gateway Time-out for url: https://mixtral.us-west-2.mesolitica.com/


 26%|█████████████████████████▊                                                                           | 1942/7596 [6:21:57<26:43:58, 17.02s/it]